## Trainning (Not using TFRecords)

### What you will see
Basically how to train a network on tensorflow.
1. Model definition
2. Loss definition
3. Session build

In [ ]:
import tensorflow as tf
import scipy.misc
import sys
sys.path.append('../')
import model
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as plt_anim
from PIL import Image
import random
import os
import subprocess
import glob
from driving_data import HandleData
from graphviz import Digraph


# Regularization value
L2NormConst = 0.001
start_lr = 0.001
batch_size=100
epochs = 600
input_train_hdf5 = '../DatasetLMDB'
input_val_hdf5 = ''
logs_path = '../logs'
save_dir = '../save'
iter_disp = 10

os.environ["CUDA_VISIBLE_DEVICES"] = str(0)

### Define the model

In [ ]:
# Open Model
driving_model = model.DrivingModel()
    
# Get placeholders from model
model_in = driving_model.input
model_out = driving_model.output
labels_in = driving_model.label_in
model_drop = driving_model.dropout_control

### Define saver object to save checkpoints

In [ ]:
# Define saver object to save all the variables of the drivingModel graph
saver = tf.train.Saver(max_to_keep=None)

### Create the Session
Basically ask tensorflow to build the graph

In [ ]:
# Avoid allocating the whole memory
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
sess = tf.InteractiveSession(config=tf.ConfigProto(gpu_options=gpu_options))

### Define Loss function

In [ ]:
train_vars = list_fc_linear
with tf.name_scope("MSE_Loss_L2Reg"):
    loss = tf.reduce_mean(tf.square(tf.subtract(labels_in, model_out))) + tf.add_n(
        [tf.nn.l2_loss(v) for v in train_vars]) * L2NormConst

# Add model accuracy
with tf.name_scope("Loss_Validation"):
    loss_val = tf.reduce_mean(tf.square(tf.subtract(labels_in, model_out)))

### Define the solver
We want to use the Adam solver to minimize or loss function.

In [ ]:
# Solver configuration
# Get ops to update moving_mean and moving_variance from batch_norm
# Reference: https://www.tensorflow.org/api_docs/python/tf/contrib/layers/batch_norm
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.name_scope("Solver"):
    global_step = tf.Variable(0, trainable=False)
    starter_learning_rate = start_lr
    # decay every 10000 steps with a base of 0.96
    learning_rate = tf.train.exponential_decay(starter_learning_rate, global_step,
                                               1000, 0.9, staircase=True)

    # Basically update the batch_norm moving averages before the training step
    # http://ruishu.io/2016/12/27/batchnorm/
    with tf.control_dependencies(update_ops):
        train_step = tf.train.AdamOptimizer(learning_rate).minimize(loss, global_step=global_step)

### Initialize the values (Random values of weights)

In [ ]:
# Initialize all random variables (Weights/Bias)
sess.run(tf.global_variables_initializer())

### Add some variables to be observed on Tensorboard

In [ ]:
# Create histogram for labels
tf.summary.histogram("steer_angle", labels_in)
# Add input image/steering angle on summary
tf.summary.image("input_image", model_in, 10)

# Monitor loss, learning_rate, global_step, etc...
tf.summary.scalar("loss_train", loss)
tf.summary.scalar("learning_rate", learning_rate)
tf.summary.scalar("global_step", global_step)
# merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

# Configure where to save the logs for tensorboard
summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

### Load the dataset

In [ ]:
data = HandleData(path=input_train_hdf5, path_val=input_val_hdf5)
num_images_epoch = int(data.get_num_images() / batch_size)
print('Num samples',data.get_num_images(), 'Iterations per epoch:', num_images_epoch, 'batch size:', batch_size)

### Do training

In [ ]:
# For each epoch
for epoch in range(epochs):
    for i in range(int(data.get_num_images() / batch_size)):
        # Get training batch
        xs_train, ys_train = data.LoadTrainBatch(batch_size, should_augment=True)

        # Send training batch to tensorflow graph (Dropout enabled)
        train_step.run(feed_dict={model_in: xs_train, labels_in: ys_train, model_drop: 0.8})

        # Display some information each x iterations
        if i % iter_disp == 0:
            # Get validation batch
            xs, ys = data.LoadValBatch(batch_size)
            # Send validation batch to tensorflow graph (Dropout disabled)
            loss_value = loss_val.eval(feed_dict={model_in: xs, labels_in: ys, model_drop: 1.0})
            print("Epoch: %d, Step: %d, Loss(Val): %g" % (epoch, epoch * batch_size + i, loss_value))

        # write logs at every iteration
        summary = merged_summary_op.eval(feed_dict={model_in: xs_train, labels_in: ys_train, model_drop: 1.0})
        summary_writer.add_summary(summary, epoch * batch_size + i)

    # Save checkpoint after each epoch
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    checkpoint_path = os.path.join(save_dir, "model")
    filename = saver.save(sess, checkpoint_path, global_step=epoch)
    print("Model saved in file: %s" % filename)

    # Shuffle data at each epoch end
    print("Shuffle data")
    data.shuffleData()